# CS 3101 - PREFINAL EXAM

## Part1: PCA and SVD from Scratch

In [38]:
def parse(file_path):
    data_started = False
    attributes = []
    data = []
    nominal_mapping = []
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()

            if not line or line.startswith('%'):
                continue

            if line.lower().startswith('@relation'):
                continue

            if line.lower().startswith('@attribute'):
                parts = line.split()
                attr_name = parts[1].strip()

                if '{' in line:
                    values = line[line.index('{') + 1:line.index('}')].split(',')
                    attributes.append((attr_name, 'nominal', values))
                    attribute_info = ((attr_name, 'nominal', values))
                    attr_name, attr_type, attr_values = attribute_info
                    nominal_mapping.append({value: index for index, value in enumerate(attr_values)})

                else:
                    attributes.append((attr_name, 'numeric', 0))

            if line.lower().startswith('@data'):
                data_started = True
                continue

            if data_started:
                data.append(line.split(','))

    return attributes, data


def standardize(matrix):
    means = [mean(col) for col in zip(*matrix)]
    std_devs = [((sum((x - means[i]) ** 2 for x in col) / len(col)) ** 0.5) for i, col in enumerate(zip(*matrix))]
    return [[(col[i] - means[i]) / std_devs[i] for i in range(len(col))] for col in matrix]

def mean(vector):
    return sum(vector) / len(vector)

def covariance(matrix):
    n = len(matrix)
    num_features = len(matrix[0])
    covariantMatrix = [[0] * num_features for _ in range(num_features)]

    for i in range(num_features):
        for j in range(num_features):
            mean_i = mean(matrix[i])
            mean_j = mean(matrix[j])
            values_i = [float(val) for val in matrix[i]]
            values_j = [float(val) for val in matrix[j]]
            covariantMatrix[i][j] = sum((val_i - mean_i) * (val_j - mean_j) for val_i, val_j in zip(values_i, values_j)) / (n - 1)

    return covariantMatrix

def normalize(vector):
    norm = sum(x**2 for x in vector)**0.5
    return [x / norm for x in vector]

def multiply(matrix, vector):
    return [sum(x*y for x, y in zip(row, vector)) for row in matrix]

def eigenVectVal(matrix, num_simulations=1000):
    n = len(matrix)
    vec = [1] * n

    for _ in range(num_simulations):
        new_vec = multiply(matrix, vec)
        vec = normalize(new_vec)

    eigenvalue = sum(x*y for x, y in zip(multiply(matrix, vec), vec))
    eigenvector = vec

    return eigenvalue, eigenvector

def transform(matrix, eigenvectors, k):
    return [
        [sum(row[j] * eigenvectors[i][j] for j in range(len(row))) for i in range(k)]
        for row in matrix
    ]
    
def pca(matrix, k):
    num_features = len(matrix[0])
    standardized_matrix= standardize(matrix)
    covariantMatrix = covariance(standardized_matrix)
    eigenvalues, eigenvectors = eigenVectVal(covariantMatrix)
    sorted_indices = sorted(range(num_features), key=lambda k: eigenvalues, reverse=True)
    eigenvectors = [[eigenvectors[i] for j in sorted_indices] for i in range(num_features)]
    transformed_matrix = transform(standardized_matrix, eigenvectors, k)
    return transformed_matrix 


def subtractVector(vevtor1, vector2):
    return [x - y for x, y in zip(vevtor1, vector2)]

def transposeMatrix(matrix):
    return [[row[i] for row in matrix] for i in range(len(matrix[0]))]

def mutilpySVD(mult1, mult):
    return [[dotProd(row, col) for col in transposeMatrix(mult)] for row in mult1]

def dotProd(vevtor1, vector2):
    return sum(x * y for x, y in zip(vevtor1, vector2))
    
def scalarMultiply(scalar, vec):
    return [scalar * x for x in vec]

def svd(data):
    meanVector = [sum(feature) / len(data) for feature in transposeMatrix(data)]
    middleData = [subtractVector(row, meanVector) for row in data]

    covariantMatrix = mutilpySVD(transposeMatrix(middleData), middleData)

    singleVector = [1.0] * len(covariantMatrix[0])
    for _ in range(50):
        singleVector = mutilpySVD([singleVector], covariantMatrix)[0]
        magnitude = sum(x ** 2 for x in singleVector) ** 0.5
        singleVector = scalarMultiply(1.0 / magnitude, singleVector)

    u_mat = [singleVector]
    v_mat = [singleVector]

    singular_val = dotProd(middleData[0], singleVector)
    s_mat = [[singular_val if i == j else 0.0 for j in range(len(u_mat))] for i in range(len(v_mat))]

    return u_mat, s_mat, transposeMatrix(v_mat)




def main():
    dataset_files = ['./samples/2019.arff',
                     './samples/2018.arff',
                    './samples/2017.arff' ]


    for file_path in dataset_files:
        dataset_label = file_path.split('/')[-1].split('.')[0]

        print(f"\n Year dataset: {dataset_label}")
        attrs, data = parse(file_path)

        for row in data:
            for i in range(len(attrs)):
                attr_name, attr_type, attr_values = attrs[i]
                if attr_type == 'nominal':
                    nom_mapping = {value: index for index, value in enumerate(attr_values)}
                    row[i] = nom_mapping.get(row[i])
                elif attr_type == 'numeric':
                    try:
                        row[i] = float(row[i])
                    except Exception as e:
                        row[i] = -1

        data_as_list = [list(map(float, row)) for row in data]

        numPCA = 5
        transformed_data = pca(data, numPCA)

    
        print(f"\nTransformed Data Size: {len(transformed_data)}")
        print("\nTransformed Data (after PCA):")
        for sample in transformed_data:
            print(sample)


        
        u_svd, s_svd, v_t_svd = svd(data_as_list)

        print("\nMatrix U (SVD):")
        for row in u_svd:
            print(row)

        print("\nMatrix S (SVD):")
        for row in s_svd:
            print(row)

        print("\nMatrix V^T (SVD):")
        for row in v_t_svd:
            print(row)

if __name__ == "__main__":
    main()


 Year dataset: 2019

Transformed Data Size: 450

Transformed Data (after PCA):
[-2.0378193926786263, -2.3838282984713985, -2.423858885760736, -2.7093525295941148, -2.472062552465901]
[-1.8955448249024986, -2.2173964046362373, -2.2546321738347896, -2.5201934895505933, -2.299470402037238]
[-1.8275332998628118, -2.137836950744355, -2.173736712787627, -2.429769775815286, -2.216966160105518]
[-1.7228888610446316, -2.015424544900802, -2.0492686888861145, -2.2906413152467957, -2.0900228208402076]
[-1.8861255616082413, -2.206377809724306, -2.2434285484717513, -2.5076702478320696, -2.288043968396844]
[-1.8814159299611126, -2.20086851226834, -2.2378267357902324, -2.501408626972808, -2.2823307515766467]
[-1.6749806081880256, -1.9593817722683446, -1.9922848144539862, -2.22694560868329, -2.0319056990450295]
[-1.5174266144955477, -1.7750761021728092, -1.804884179750718, -2.0174720346796255, -1.8407781981497393]
[-1.6052965089730713, -1.8778657516342752, -1.9093999308940235, -2.134298148776475, -1.9

### Explanation: